In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
import os
import sys
import numpy as np
from itertools import product
import logging
import tqdm
from sklearn.metrics import average_precision_score, roc_auc_score
sys.path.append(os.path.abspath(os.path.join('..', 'src',)))

from interfaces.classifier import Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier

from models.classifiers.hierarchical import Hierarchical
import h2o
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from models.classifiers.h2o import H2OMultiLabel

h2o.init(verbose=False)

/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
outputs_df = pd.read_parquet("../data/processed/outputs_openai_embeddings_v1.parquet")
ppas = outputs_df.PPAs_list.explode().drop_duplicates().sort_values().to_list()
train_df = outputs_df[~outputs_df.test_set]
test_df = outputs_df[outputs_df.test_set]
X_labels = outputs_df.columns[outputs_df.columns.str.contains("openai_embedding_small")]
# y_label = "PPAs_list"
oversampling = True

X_train = train_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
y_train = train_df[ppas].astype(int).to_numpy()

X_test = test_df[X_labels].to_numpy()
# y_test = test_df[y_label].to_numpy()
y_test = test_df[ppas].astype(int).to_numpy()


ros = RandomOverSampler(random_state=42)
train_oversamples_df, _ = ros.fit_resample(train_df, y=train_df["primary_ppa"])
X_train_oversampled = train_oversamples_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
y_train_oversampled = train_oversamples_df[ppas].astype(int).to_numpy() 


def ppa_hierarchy(ppas_list: pd.Series):
    return ppas_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()


def experiment(name: str, clf: Classifier, X_train: np.ndarray, y_train: np.ndarray, 
               X_test: np.ndarray, y_test_df: pd.DataFrame, classes: list[str]):
    clf.fit(X_train, y_train)
    y_pred_proba = clf.predict_proba(X_test)
    y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=classes)

    experiment = []
    for ppa in ppas:
        y_test_ppa = y_test_df[ppa]
        experiment.append({
            "name": name,
            "ppa": ppa,
            "roc_auc": roc_auc_score(y_test_ppa, y_pred_proba_df[ppa]),
            "average_precision": average_precision_score(y_test_ppa, y_pred_proba_df[ppa]),
        })
    return experiment

class CustomDecisionTree(DecisionTreeClassifier):
    def predict_proba(self, X):
        probs = super().predict_proba(X)
        return np.array(probs)[:, :, 0].T

In [5]:
from models.classifiers.ridge import CustomRidge


clfs = [
    # (H2OMultiLabel(max_models=1, seed=1), "H2O MultiLabel"),
    (MLPClassifier(max_iter=1000), "MLP_1000it"),
    # (RandomForestClassifier(), "Random Forest"),
    (CustomRidge(), "Ridge")
]

all_experiments = []

# experiments = list(product([True, False], [True, False], clfs))
oversampling = True

for clf, name in tqdm.tqdm(clfs):
    train_df_exp = train_oversamples_df if oversampling else train_df
    
    X_train_exp = X_train_oversampled if oversampling else X_train
    y_train_exp = y_train_oversampled if oversampling else y_train
    
    X_test_exp = X_test
    y_test_exp_df = test_df[ppas].astype("int")
    
    if isinstance(clf, Hierarchical):
        # clf = Hierarchical(sklearn_clf, ppas)
        y_train_exp = train_df_exp.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()
        y_test_exp = test_df.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()    
    
    try:
        experiment_result = experiment(name, clf, X_train_exp, y_train_exp, X_test_exp, y_test_exp_df, ppas)
        all_experiments.extend(experiment_result)
    except Exception as e:
        logging.error(f"Error in {((clf, name))}: {repr(e)}")
        
experiments_df = pd.DataFrame(all_experiments)
experiments_df.groupby("name")[["roc_auc", "average_precision"]].agg(["mean","std"]).sort_values(("roc_auc", "mean"), ascending=False)

  0%|          | 0/2 [00:00<?, ?it/s]/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 2/2 [00:05<00:00,  2.69s/it]


roc_auc           average_precision          
                mean       std              mean       std
name                                                      
Ridge       0.895644  0.122542          0.616983  0.290054
MLP_1000it  0.873290  0.157631          0.552559  0.279793

In [53]:
from pathlib import Path


clfs[0][0].clfs[0].clf.leader.save_mojo("here")

'/Users/jm/Code/fao-ppa-classifier/notebooks/here/GLM_1_AutoML_52_20240610_223417.zip'

In [50]:
clfs[0][0].clfs[0].clf

In [47]:
clfs[0][0].predict_proba(X_test).shape

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |

/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |

/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |

...

████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |

/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


(40, 20)

In [32]:
experiments_df

""


In [ ]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
# train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
# test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")
# h2o.Data

train_h2o_df = h2o.H2OFrame(train_df)
test_h2o_df = h2o.H2OFrame(test_df)

# Identify predictors and response
# x = train.columns
# y = "response"
# x.remove(y)

# For binary classification, response should be a factor
# train[y] = train[y].asfactor()
# test[y] = test[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=2, seed=1)
aml.train(x=X_labels.to_list(), y="BP.1", training_frame=train_h2o_df)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

In [ ]:
train_h2o_df

In [ ]:
import numpy as np

# aa = h2o.H2OFrame(X_train)
# aml.train(x=X_labels.to_list(), y="BP.1", training_frame=train_h2o_df)
# aa
h2o.init()


In [ ]:
X_df = pd.DataFrame(X_train).add_prefix("X_")
y_df = pd.DataFrame(y_train).add_prefix("y_").astype("int")
                                
y_col = y_df.columns[0]
train_h2o_df = h2o.H2OFrame(X_df.join(y_df))
train_h2o_df[y_col] = train_h2o_df[y_col].asfactor()

# test_h2o_df = h2o.H2OFrame(test_df)

# Identify predictors and response
# x = train.columns
# y = "response"
# x.remove(y)

# For binary classification, response should be a factor
# train[y] = train[y].asfactor()
# test[y] = test[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=2, seed=1)
aml.train(x=X_df.columns.to_list(), y=y_df.columns[0], training_frame=train_h2o_df)

lb = aml.leaderboard
lb.head(rows=lb.nrows) 

In [ ]:

c.fit(X_train, y_train)
roc_auc_score(test_df[ppas[0]].astype("int").to_numpy(), c.predict(X_test))

In [ ]:
aml.predict(h2o.H2OFrame(test_df_2)).as_data_frame()

In [ ]:
roc_auc_score(test_df[ppas[0]].astype("int").to_numpy(), aml.predict(h2o.H2OFrame(test_df_2)).as_data_frame()["p1"].to_numpy())

In [ ]:
average_precision_score(test_df[ppas[0]].astype("int").to_numpy(), aml.predict(h2o.H2OFrame(test_df_2)).as_data_frame()["p1"].to_numpy())

In [ ]:
# test_h2o_df2 = test_h2o_df
# test_h2o_df2.columns = X_df.columns.to_list()
aml.predict(h2o.H2OFrame(test_df_2)).as_data_frame()["p1"]

aml.predict(h2o.H2OFrame(test_df_2)).as_data_frame()["p1"]

In [ ]:
aml.predict

In [ ]:
y_df.sum().sum()

In [ ]:
from cgi import test


train_df["BP.1"].to_numpy()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression


LogisticRegression
OneVsRestClassifier()

In [ ]:
roc_auc_score(test_df["BP.1"].astype("int").to_numpy(), aml.predict(test_h2o_df)["True"].as_data_frame().to_numpy())

In [ ]:
average_precision_score(test_df["BP.1"].astype("int").to_numpy(), aml.predict(test_h2o_df)["True"].as_data_frame().to_numpy())

In [ ]:
experiments_df

In [ ]:
class CustomRidge(RidgeClassifier):
    def predict_proba(self, X):
        scores = self.decision_function(X)
        y_pred_proba = np.exp(scores) / np.sum(np.exp(scores))
        return y_pred_proba




clfs = [
    (MLPClassifier(), MLPClassifier(), "MLP_200it"),
    (MLPClassifier(max_iter=1000), MLPClassifier(max_iter=1000), "MLP_1000it"),
    (RandomForestClassifier(),RandomForestClassifier(), "Random Forest"),
    (CustomRidge(), RidgeClassifier(), "Ridge")
]

all_experiments = []


experiments = list(product([True, False], [True, False], clfs))

for hierarchical, oversampling, (clf, sklearn_clf, name) in tqdm.tqdm(experiments):
    train_df_exp = train_oversamples_df if oversampling else train_df
    
    X_train_exp = X_train_oversampled if oversampling else X_train
    y_train_exp = y_train_oversampled if oversampling else y_train
    
    X_test_exp = X_test
    y_test_exp_df = test_df[ppas].astype("int")
    
    if hierarchical:
        clf = Hierarchical(sklearn_clf, ppas)
        y_train_exp = train_df_exp.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()
        y_test_exp = test_df.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()    
    
    try:
        experiment_result = experiment(name, clf, X_train_exp, y_train_exp, X_test_exp, y_test_exp_df, ppas, hierarchical, oversampling)
        all_experiments.extend(experiment_result)
    except Exception as e:
        logging.error(f"Error in {(hierarchical, oversampling, (clf, sklearn_clf, name))}: {repr(e)}")
        
experiments_df = pd.DataFrame(all_experiments)
experiments_df.groupby(["name", "hierarchical", "oversampling"])[["roc_auc", "average_precision"]].agg(["mean","std"]).sort_values(("roc_auc", "mean"), ascending=False)

In [ ]:
class CustomRidge(RidgeClassifier):
    def predict_proba(self, X):
        scores = self.decision_function(X)
        y_pred_proba = np.exp(scores) / np.sum(np.exp(scores))
        return y_pred_proba


class CustomRandomForest(RandomForestClassifier):
    def predict_proba(self, X):
        probs = super().predict_proba(X)
        return np.array(probs)[:, :, 0].T




clfs = [
    # (MLPClassifier(), MLPClassifier(), " £sifier(max_iter=1000), "MLP_1000it"),
    # (CustomRandomForest(),RandomForestClassifier(), "Random Forest"),
    # (CustomRidge(), RidgeClassifier(), "Ridge")
    (MLPClassifier(max_iter=10_000), MLPClassifier(max_iter=1000), "MLP_1000it"),
]

all_experiments = []



experiments = list(product([True, False], [True, False], clfs))

for hierarchical, oversampling, (clf, sklearn_clf, name) in tqdm.tqdm(experiments):
    train_df_exp = train_oversamples_df if oversampling else train_df
    
    X_train_exp = X_train_oversampled if oversampling else X_train
    y_train_exp = y_train_oversampled if oversampling else y_train
    
    X_test_exp = X_test
    y_test_exp_df = test_df[ppas].astype("int")
    
    if hierarchical:
        clf = Hierarchical(sklearn_clf, ppas)
        y_train_exp = train_df_exp.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()
        y_test_exp = test_df.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()    
    
    try:
        experiment_result = experiment(name, clf, X_train_exp, y_train_exp, X_test_exp, y_test_exp_df, ppas, hierarchical, oversampling)
        all_experiments.extend(experiment_result)
    except Exception as e:
        logging.error(f"Error in {(hierarchical, oversampling, (clf, sklearn_clf, name))}: {repr(e)}")
        
experiments_df = pd.DataFrame(all_experiments)
experiments_df.groupby(["name", "hierarchical", "oversampling"])[["roc_auc", "average_precision"]].agg(["mean","std"]).sort_values(("roc_auc", "mean"), ascending=False)

In [ ]:
experiments_df.groupby(["name", "hierarchical", "oversampling"])[["roc_auc", "average_precision"]].agg(["mean","std"]).sort_values(("roc_auc", "mean"), ascending=False)